# Run Pipeplines with Azure Machine Learning Python SDK

You use the Azure python SDK to orchestrate steps in a Pipeline will run in succession or in Parallel on a compute target.

You'll need the latest version of the **azure.ai.ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure.ai.ml** package is not installed, run `pip install azure.ai.ml` to install it.

In [ ]:
%pip show azure.ai.ml

# Connect to a Workspace

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. A `config.json` file containing these parameters can be downloaded from the Azure Machine Learning workspace or Azure portal.

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential= DefaultAzureCredential(), path="../../config.json")


Found the config file in: ../../config.json


## Register the dataset

Azure Machine Learning providers several datastores that encapsulates a Dataset. Be considerate about the kind of datastores, use cases, and associated costs to determine the best datasource. Here we use the default datasource which is `blob` data store.

**Authenticate with `Azure CLI` is required here**

## Register the dataset

Azure Machine Learning providers several datastores that encapsulates a Dataset. Be considerate about the kind of datastores, use cases, and associated costs to determine the best datasource. Here we use the default datasource which is `blob` data store.

**Authenticate with `Azure CLI` is required here**

In [2]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

file = "../../data/diabetes.csv"

diabetes_data = Data(
    name="diabetes_csv",
    path=file,
    type=AssetTypes.URI_FILE,
    description="Dataset for Diabetes model training",
    tags={"source_type": "file", "source": "Local file"},
    version="1.0.0",
)

try:
    diabetes_data = ml_client.data.create_or_update(diabetes_data)
except (Exception) as ex:  
    print("Exception while registering dataset ", ex)

Exception while registering dataset  (UserError) A data version with this name and version already exists. If you are trying to create a new data version, use a different name or version. If you are trying to update an existing data version, the existing asset's data uri cannot be changed. Only tags, description, and isArchived can be updated.
Code: UserError
Message: A data version with this name and version already exists. If you are trying to create a new data version, use a different name or version. If you are trying to update an existing data version, the existing asset's data uri cannot be changed. Only tags, description, and isArchived can be updated.
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "f50768dc8433c56720f70ba5ac968802",
        "request": "09a2023dc8713835"
    }
}Type: Environment
Info: {
    "value": "northeurope"
}Type: Location
Info: {
    "value": "northeurope"

# Create Component dirs

In [3]:
import os
components_dirs = {
      "eda":  "./components/eda",
      "data_prep": "./components/data_prep",
      "train": "./components/train"
}

for (key, value) in components_dirs.items():
    os.makedirs(value, exist_ok=True)

## Exploratory data analysis script

Plot correlation, feature-wise distributions and pairwise scatter plots

In [4]:
%%writefile {components_dirs["eda"]}/diabetes-exploratory-plots.py
# Plot distrubtions step

import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
from itertools import combinations
import argparse

# Create a function that we can re-use
def plot_correlations(data):
    """
    This function will make a correlation graph and save it
    """
    correlation = data.corr()
    print("Correlation between features\n", correlation)

    fig = plt.figure(figsize=(10, 12))
    sns.heatmap(data=correlation, annot=True)
    plt.title("Correlation betweeen features")

    # Save plot
    filename = "outputs/correlations-between-features.png"
    os.makedirs("outputs", exist_ok=True)
    fig.savefig(filename)


def plot_distribution(var_data, column_name=None):
    """
    This function will make a distribution (graph) and save it
    """

    # Get statistics
    min_val = var_data.min()
    max_val = var_data.max()
    mean_val = var_data.mean()
    med_val = var_data.median()
    mod_val = var_data.mode()[0]

    print(
        "{} Statistics:\nMinimum:{:.2f}\nMean:{:.2f}\nMedian:{:.2f}\nMode:{:.2f}\nMaximum:{:.2f}\n".format(
            "" if column_name is None else column_name,
            min_val,
            mean_val,
            med_val,
            mod_val,
            max_val,
        )
    )

    # Create a figure for 2 subplots (2 rows, 1 column)
    fig, ax = plt.subplots(2, 1, figsize=(10, 4))

    # Plot the histogram
    ax[0].hist(var_data)
    ax[0].set_ylabel("Frequency")

    # Add lines for the mean, median, and mode
    ax[0].axvline(x=min_val, color="gray", linestyle="dashed", linewidth=2)
    ax[0].axvline(x=mean_val, color="cyan", linestyle="dashed", linewidth=2)
    ax[0].axvline(x=med_val, color="red", linestyle="dashed", linewidth=2)
    ax[0].axvline(x=mod_val, color="yellow", linestyle="dashed", linewidth=2)
    ax[0].axvline(x=max_val, color="gray", linestyle="dashed", linewidth=2)
    ax[0].legend()

    # Plot the boxplot
    ax[1].boxplot(var_data, vert=False)
    xlabel = "Value" if column_name is None else column_name
    ax[1].set_xlabel(xlabel)

    # Add a title to the Figure
    title = (
        "Data Distribution"
        if column_name is None
        else "{} Data Distribution".format(column_name)
    )
    fig.suptitle(title)

    # Save plot
    filename = "outputs/{}-distribution.png".format(column_name)
    os.makedirs("outputs", exist_ok=True)
    fig.savefig(filename)


def plot_scatters(x_y_data):
    """
    Plot scatter plots with :y_column: on y-axis and save them. 
    """
    
    x_column = x_y_data.columns.values[0]
    y_column = x_y_data.columns.values[1]

    fig = plt.figure(figsize=(10, 12))
    sns.regplot(data=x_y_data,x=x_column, y=y_column)
    plt.xlabel(x_column)
    plt.ylabel(y_column)
    plt.title("Scatter plot of {} vs {}".format(x_column,y_column))

    # Save plot
    filename = "outputs/Scatter plot of {} vs {}.png".format(x_column,y_column)
    os.makedirs("outputs", exist_ok=True)
    fig.savefig(filename)

def main():
    print("Loading Data...")

    parser = argparse.ArgumentParser()
    parser.add_argument('--data-set', type=str,dest="data")
    args = parser.parse_args()

    diabetes = pd.read_csv(args.data, header= 0)

    # plot correlations
    plot_correlations(data=diabetes)

    # plot distributions
    exlude_column = set(["Diabetic", "PatientID"])
    columns = diabetes.columns.values
    for x in columns:
        if x not in exlude_column:
            plot_distribution(var_data=diabetes[x],column_name=x)

    # plot scatter plots
    columns = set(columns)


    column_comb=list(combinations(columns-exlude_column,2))
    column_comb = [list(x) for x in column_comb]

    for x_y_pairs in column_comb:
        plot_scatters(diabetes[x_y_pairs])

if __name__ == "__main__":
    main()

Overwriting ./components/eda/diabetes-exploratory-plots.py


# Data preparation script


In [5]:
%%writefile {components_dirs["data_prep"]}/data_prep.py
import os
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split

def main():
    # Parse job parameters
    parser = argparse.ArgumentParser()
    parser.add_argument('--data', type=str)
    parser.add_argument('--test_size', type=float, default=0.30)
    parser.add_argument('--train_data', type=str)
    parser.add_argument('--test_data', type=str)
    args = parser.parse_args()

    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    data = pd.read_csv(args.data, header=0)
    print("num_samples:", data.shape[0])
    print("num_features", data.shape[1] - 1)

    train_data, test_data,  = train_test_split(data, test_size=args.test_size, random_state=0)

    # args are mounted folders, so write the data into the folder
    train_data.to_csv(os.path.join(args.train_data,"train_data.csv"))
    test_data.to_csv(os.path.join(args.test_data,"test_data.csv"))
    
if __name__ == "__main__":
    main()

Overwriting ./components/data_prep/data_prep.py


## Model Training script

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. 

In [34]:
%%writefile {components_dirs["train"]}/diabetes-training.py
# import libraries
import os
import argparse
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

def main():

    # Parse job parameters
    parser = argparse.ArgumentParser()
    parser.add_argument('--reg_rate', type=float, default=0.01)
    parser.add_argument('--train_data', type=str, help="path to train data")
    parser.add_argument('--test_data', type=str, help="path to test data")
    parser.add_argument("--trained_model", type=str,help="path to trained data")
    args = parser.parse_args()


    # load the diabetes dataset
    print("Loading Data...")
    train_file = os.path.join(args.train_data, "train_data.csv")
    test_file = os.path.join(args.test_data, "test_data.csv")
    train_df = pd.read_csv(train_file, header=0, index_col="PatientID")
    y_train = train_df.pop("Diabetic")
    x_train = train_df.values
    test_df = pd.read_csv(test_file, header=0, index_col="PatientID")
    y_test = test_df.pop("Diabetic")
    x_test = test_df.values

    print("num_train_samples:", x_train.shape[0])
    print("num_test_samples:", x_test.shape[0])
    print("num_features:", x_train.shape[1])
    print("features:", x_train.columns.values)


    # train a logistic regression model
    print('Training a logistic regression model with regularization rate of', args.reg_rate)
    model = LogisticRegression(C=1/args.reg_rate, solver="liblinear").fit(x_train, y_train)

    # calculate accuracy
    y_hat = model.predict(x_test)
    acc = np.average(y_hat == y_test)
    print('Accuracy:', float(acc))

    # calculate AUC
    y_scores = model.predict_proba(x_test)
    auc = roc_auc_score(y_test,y_scores[:,1])
    print('AUC:' + str(auc))

    # Save the model to file
    print("Saving model to file")
    filename = os.path.join(args.trained_model,"model.pkl")
    os.makedirs('outputs', exist_ok=True)
    with open(filename, 'wb') as file:
        pickle.dump(model,file)


if __name__ == "__main__":
    main()

Overwriting ./components/train/diabetes-training.py


# Create Environment 

In [7]:
import os
dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [8]:
%%writefile {dependencies_dir}/conda.yaml
name: sklearn-1.5
channels:
- conda-forge
- anaconda
dependencies:
- python=3.10
- pip=21.3.1
- pandas~=1.5.3
- scipy~=1.10.0
- numpy~=1.22.0
- pip:
  - scikit-learn-intelex==2024.6.0
  - azureml-core==1.57.0
  - azureml-defaults==1.57.0
  - azureml-mlflow==1.57.0.post1
  - azureml-telemetry==1.57.0
  - scikit-learn~=1.5.0
  - joblib~=1.2.0
  # azureml-automl-common-tools packages
  - py-spy==0.3.12
  - debugpy~=1.6.3
  - ipykernel~=6.0
  - tensorboard
  - psutil~=5.8.0
  - matplotlib~=3.5.0
  - tqdm~=4.66.3
  - py-cpuinfo==5.0.0
  - torch-tb-profiler~=0.4.0


Overwriting ./dependencies/conda.yaml


In [9]:
from azure.ai.ml.entities import Environment

env_name = "diabetes-scikit-learn"

env = Environment(
    name=env_name,
    description="Custom environment for Diabetes prediction pipeline",
    tags={"scikit-learn": "1.5.0"},
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest"
)
env = ml_client.environments.create_or_update(env)

print(
    f"Environment with name {env.name} is registered to workspace, the environment version is {env.version}"
)

Environment with name diabetes-scikit-learn is registered to workspace, the environment version is 2


# Create pipeline components

In [19]:
command_env="{0}:{1}".format(env.name,env.version)
command_env

'diabetes-scikit-learn:2'

In [35]:
from azure.ai.ml import command, Input, Output

command_env = "{0}:{1}".format(env.name, env.version)
experiment_name = "diabetes-training"

data_prep_command = command(
    name="data_prep_diabetes",
    display_name="Data preparation for training",
    description="Read the diabetes csv data and splits into train and test sets",
    inputs=dict(data=Input(type="uri_file"), test_size=Input(type="number")),
    outputs=dict(
        train_data=Output(type="uri_folder", mode="rw_mount"),
        test_data=Output(type="uri_folder", mode="rw_mount"),
    ),
    code=components_dirs["data_prep"],
    command="python data_prep.py  --data ${{inputs.data}} --test_size ${{inputs.test_size}}  --train_data ${{outputs.train_data}} --test_data ${{outputs.test_data}}",
    environment=command_env,
    experiment_name=experiment_name,
)

# Now we register the component to the workspace
data_prep_component = ml_client.create_or_update(data_prep_command.component)

# Create (register) the component in your workspace
print(
    f"Component {data_prep_component.name} with Version {data_prep_component.version} is registered"
)

eda_command = command(
    name="eda_diabetes",
    display_name="Exploratory data analysis",
    description="Reads the diabetes csv data and plot exploratory data analysis graphs",
    inputs=dict(data=Input(type="uri_file")),
    code=components_dirs["eda"],
    command="python diabetes-exploratory-plots.py --data ${{inputs.data}}",
    environment=command_env,
    experiment_name=experiment_name,
)

# Now we register the component to the workspace
eda_command_component = ml_client.create_or_update(eda_command.component)

# Create (register) the component in your workspace
print(
    f"Component {eda_command_component.name} with Version {eda_command_component.version} is registered"
)

model_train_command = command(
    name="model_train_diabetes",
    display_name="Train model",
    description="Train a Logistic Regression model diabetes dataset",
    inputs=dict(
        train_data=Input(type="uri_folder", mode="rw_mount"),
        test_data=Input(type="uri_folder", mode="rw_mount"),
        reg_rate=Input(type="number"),
    ),
    outputs=dict(trained_model=Output(type="uri_folder", mode="rw_mount")),
    code=components_dirs["train"],
    command="python diabetes-training.py --train_data ${{inputs.train_data}} --test_data ${{inputs.test_data}} --reg_rate ${{inputs.reg_rate}} --trained_model ${{outputs.trained_model}}",
    environment=command_env,
    experiment_name=experiment_name,
)

# Now we register the component to the workspace
model_train_command_component = ml_client.create_or_update(model_train_command.component)

# Create (register) the component in your workspace
print(
    f"Component {model_train_command_component.name} with Version {model_train_command_component.version} is registered"
)

Component data_prep_diabetes with Version 2024-09-15-08-42-32-0349685 is registered
Component eda_diabetes with Version 2024-09-15-08-42-36-3069890 is registered


Uploading train (0.0 MBs): 100%|██████████| 1954/1954 [00:00<00:00, 25285.60it/s]




Component model_train_diabetes with Version 1 is registered


## Building the Pipeline

The `enviroment` was created in Azure ML workspace, but can be created with a script.

Test data size (`test_size`) and Regularization rate (`reg_rate`) for the `LogisticRegression` are passed as parameters. Other parameters such as a registered dataset in a data store could also be passed.

In [ ]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes

data_asset = ml_client.data.get("diabetes_csv", version="1.0.0")

# Define arguments / parameters
diabetes_data = ml_client.data.get("diabetes_csv", version="1.0.0")
test_size = 0.30
reg_rate = 0.01

run_command = command(
    code="./src",
    command="python diabetes-training.py --data ${{inputs.data}} --test-size ${{inputs.test_size}} --reg-rate ${{inputs.reg_rate}} ",
    inputs=dict(
        data= Input(
            path=diabetes_data.id,
            type=AssetTypes.URI_FILE,
            mode=InputOutputModes.RO_MOUNT,
        ),
        reg_rate = reg_rate,
        test_size = test_size,
    ),
    environment="diabest-train:8",
    experiment_name = "diabetes-training"
)

returned_job = ml_client.jobs.create_or_update(run_command)

In [31]:
import pandas as pd

df = pd.read_csv("../../data/diabetes.csv", index_col="PatientID")

y = df.pop("Diabetic")

In [32]:
y.head()

PatientID
1354778    0
1147438    0
1640031    0
1883350    1
1424119    0
Name: Diabetic, dtype: int64

In [33]:
df.head()

,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age
PatientID,,,,,,,,
1354778,0,171,80,34,23,43.509726,1.213191,21
1147438,8,92,93,47,36,21.240576,0.158365,23
1640031,7,115,47,52,35,41.511523,0.079019,23
1883350,9,103,78,25,304,29.582192,1.282870,43
1424119,1,85,59,27,35,42.604536,0.549542,22
